In [97]:
#dependencies
from pint import UnitRegistry
import numpy as np
import pandas as pd


#unit setup
ureg = UnitRegistry()
m = ureg.meter
g = ureg.gram
kg = ureg.kilogram
s = ureg.second
min = ureg.minute
rpm = ureg.revolution / min
turns = ureg.turn
W = ureg.watt
N = ureg.newton
K = ureg.kelvin
#variable array setup
speed = ureg.Quantity(np.array([1500, 2000, 2500, 3000, 3500, 4000]), rpm)
torque = ureg.Quantity(np.array([1.8, 1.8, 1.8, 1.8, 1.8, 1.8]), N * m)
Qdot_shaft = ureg.Quantity(np.array([None, None, None, None, None, None]), W) # Qdot_shaft
Qdot_in = ureg.Quantity(np.array([None, None, None, None, None, None]), W) # heat input
efficiency = np.array([None, None, None, None, None, None]) # n_th percent
MEP = ureg.Quantity(np.array([None, None, None, None, None, None]), N / m**2) # mean effective pressure
bsfc = ureg.Quantity(np.array([None, None, None, None, None, None]), g / (W * s)) # brake specific fuel consumption
Qdot_exhaust = ureg.Quantity(np.array([None, None, None, None, None, None]), W) # exhaust heat
Qdot_fins = ureg.Quantity(np.array([None, None, None, None, None, None]), W) # fins heat
mdot_air = ureg.Quantity(np.array([None, None, None, None, None, None]), kg/s) # mass flow rate of air
mdot_fuel = ureg.Quantity(np.array([5.55E-05, 5.64E-05, 6.83E-05, 8.26E-05, 0.000104, 0.000118]), kg / s) # mass flow rate of fuel
deltaT = ureg.Quantity(np.array([247, 247, 247, 291, 330, 350]), K) # temperature difference

In [101]:
#constants
rho_air = 1.2 * ureg.kilogram / ureg.meter**3
Cp_air = 1.006 * ureg.kilojoule / (ureg.kilogram * ureg.kelvin)
#calculations
D = 10e-4 * m**3 # displacement of the engine
mdot_air = (rho_air * D * speed) / (2*turns) # The 2 is present in the denominator because engine will only draw air every second revolution
mdot_air = mdot_air.to(kg / s)
print(mdot_air)
Qdot_exhaust = mdot_air * Cp_air * deltaT
print(Qdot_exhaust)
Qdot_shaft = speed*torque
Qdot_shaft = Qdot_shaft.to(W)
print(Qdot_shaft)

# mdot_gas = (rho_gas * deltaV) / deltat
# Qdot_in = mdot_gas * LHV_gas
# Qdot_in = Qdot_exhaust + Qdot_shaft + Qdot_fins
# efficiency = Qdot_shaft / Qdot_in
# bsfc = mdot_fuel / Qdot_shaft
# MEP = 4 * np.pi *(torque / D)

[0.015 0.02  0.025 0.03  0.035 0.04 ] kilogram / second
[ 3.72723  4.96964  6.21205  8.78238 11.6193  14.084  ] kilojoule / second
[282.74333882 376.99111843 471.23889804 565.48667765 659.73445725 753.98223686] watt


In [99]:
# Create a dictionary with the column names and data
data = {
    'Speed (RPM)': speed.magnitude,
    'Torque (N m)': torque.magnitude,
    'Power Shaft (W)': power_shaft.magnitude,
    'Power Input (W)': power_input.magnitude,
    'Efficiency': efficiency,
    'MEP (N/m^2)': MEP.magnitude,
    'BSFC (g/(W*s))': bsfc.magnitude,
    'Exhaust Heat (W)': exhaust_heat.magnitude,
    'Fins Heat (W)': fins_heat.magnitude,
    'Air Mass Flow (g/s)': air_mass_flow.magnitude,
    'Fuel Mass Flow (g/s)': fuel_mass_flow.magnitude,
    'Delta T (delta_degC)': deltaT.magnitude
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Display the DataFrame in a nice table
df


,Speed (RPM),Torque (N m),Power Shaft (W),Power Input (W),Efficiency,MEP (N/m^2),BSFC (g/(W*s)),Exhaust Heat (W),Fins Heat (W),Air Mass Flow (g/s),Fuel Mass Flow (g/s),Delta T (delta_degC)
0,1500,1.8,None,0.000056,None,None,None,None,None,None,None,247
1,2000,1.8,None,0.000056,None,None,None,None,None,None,None,247
2,2500,1.8,None,0.000068,None,None,None,None,None,None,None,247
3,3000,1.8,None,0.000083,None,None,None,None,None,None,None,291
4,3500,1.8,None,0.000104,None,None,None,None,None,None,None,330
5,4000,1.8,None,0.000118,None,None,None,None,None,None,None,350
